In [1]:
# from skimage.util import as_float

# # import cython
# import pyximport
# pyximport.install(setup_args={"script_args" : ["--verbose"]})

# from skimage.filters import gaussian
# from skimage2.utils import _supported_float_type
# from skimage.color import rgb2lab
# from skimage.util import img_as_float
# from skimage2._quickshift_cy import _quickshift_cython

In [1]:
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import random
import rasterio
import numpy as np
import glob
from rasterio.plot import reshape_as_image
from tensorflow.keras import models

from LimeImageExplainer2 import LimeImageExplainer2


file = rasterio.open('Input/sentinel/patches_256/Iowa_July_1_31/test/Iowa_2021_july_1280-3072.tif')
img = reshape_as_image(file.read())
file.close()
img_batch = np.expand_dims(img, axis=0)
# model_id = "aanaxs4g" # With mask
model_id = "ezb3xkqf" # No Mask
model_path = glob.glob("wandb/"+ "*"+model_id+"*" + "/files/model-best.h5")[0]

# print(model_path)
cnn_model = models.load_model(model_path)

2023-03-17 14:59:55.964435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib::/usr/lib/jvm/java-11-openjdk-amd64/lib/server:/opt/hadoop/lib/native:/usr/local/lib/R/lib:/usr/local/grass82/lib
2023-03-17 14:59:55.964449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-17 14:59:57.814930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib::/usr/lib/jvm/java-11-openjdk-amd64/lib/server:/opt/hadoop/lib/native:/usr/local/lib/R/lib:/usr/local/grass82/lib
2023-03-17 14:59:57.814944: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [ ]:
explainer = LimeImageExplainer2(random_state=42)
explanation = explainer.explain_instance(
         img_batch,
         cnn_model.predict
)
# plt.imshow(img_batch[10])
# image, mask = explanation.get_image_and_mask(
#          cnn_model.predict(
#               img_batch).argmax(axis=1)[0],positive_only=True,hide_rest=False)
# plt.imshow(mark_boundaries(image, mask))

(1, 256, 256, 13)
<class 'LimeImageExplainer2.LimeImageExplainer2'>


ValueError: Cannot convert from object to float64.